In [0]:
%load_ext autoreload
%autoreload 2
# Enables autoreload; learn more at https://docs.databricks.com/en/files/workspace-modules.html#autoreload-for-python-modules
# To disable autoreload; run %autoreload 0

 DIMUSERS

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
import os
import sys
project_path = os.path.join(os.getcwd(), '..', '..')
##display(project_path)

sys.path.append(project_path)
from utils.transformations import reuseable

In [0]:
df = spark.read.format("parquet").load("abfss://bronze@anshulstorageproject.dfs.core.windows.net/DimUser")


### **AUtoLoader**

### **DimUser**

In [0]:
df_user = spark.readStream.format("cloudFiles")\
          .option("cloudFiles.format", "parquet")\
          .option("cloudFiles.schemaLocation", "abfss://silver@anshulstorageproject.dfs.core.windows.net/DimUser/checkpoint")\
          .load("abfss://bronze@anshulstorageproject.dfs.core.windows.net/DimUser")

In [0]:
# Apply transformations
df_user_obj = reuseable()
df_user = df_user_obj.dropColumns(df_user, ['_rescued_data'])

# Deduplicate by user_id
df_user = df_user.dropDuplicates(['user_id'])
#display(df_user)

In [0]:
df_user.writeStream.format("delta")\
    .outputMode("append")\
    .option("checkpointLocation", "abfss://silver@anshulstorageproject.dfs.core.windows.net/DimUser/checkpoint")\
    .trigger(once=True)\
    .start("abfss://silver@anshulstorageproject.dfs.core.windows.net/DimUser/data")

In [0]:
df_user.writeStream.format("delta")\
    .outputMode("append")\
    .option("checkpointLocation", "abfss://silver@anshulstorageproject.dfs.core.windows.net/DimUser/checkpoint")\
        .trigger(once=True)\
            .option("path", "abfss://silver@anshulstorageproject.dfs.core.windows.net/DimUser/data")\
            .toTable("spotifycatalog.silver.DimUser")

### **DimArtist**

In [0]:
df_artist = spark.readStream.format("cloudFiles")\
          .option("cloudFiles.format", "parquet")\
          .option("cloudFiles.schemaLocation", "abfss://silver@anshulstorageproject.dfs.core.windows.net/DimArtist/checkpoint")\
          .load("abfss://bronze@anshulstorageproject.dfs.core.windows.net/DimArtist")

In [0]:

# Apply transformations
df_artist_obj = reuseable()
df_artist = df_artist_obj.dropColumns(df_artist, ['_rescued_data'])

# Deduplicate by artist_id
df_artist = df_artist.dropDuplicates(['artist_id'])

In [0]:
df_artist = df_artist.repartition(8)

df_artist.writeStream.format("delta")\
    .outputMode("append")\
    .option("checkpointLocation", "abfss://silver@anshulstorageproject.dfs.core.windows.net/DimArtist/checkpoint")\
    .trigger(once=True)\
    .option("path", "abfss://silver@anshulstorageproject.dfs.core.windows.net/DimArtist/data")\
    .toTable("spotifycatalog.silver.DimArtist")

### **DimDate** 

In [0]:
df_date = spark.readStream.format("cloudFiles")\
          .option("cloudFiles.format", "parquet")\
          .option("cloudFiles.schemaLocation", "abfss://silver@anshulstorageproject.dfs.core.windows.net/DimDate/checkpoint")\
          .load("abfss://bronze@anshulstorageproject.dfs.core.windows.net/DimDate")

In [0]:
# Apply transformations
df_date_obj = reuseable()
df_date = df_date_obj.dropColumns(df_artist, ['_rescued_data'])

# Deduplicate by artist_id
df_date = df_date.dropDuplicates(['artist_id'])

In [0]:
df_date.writeStream.format("delta")\
    .outputMode("append")\
    .option("checkpointLocation", "abfss://silver@anshulstorageproject.dfs.core.windows.net/DimDate/checkpoint")\
    .trigger(once=True)\
    .option("path", "abfss://silver@anshulstorageproject.dfs.core.windows.net/DimDate/data")\
    .toTable("spotifycatalog.silver.DimDate")

### **DimTrack**

In [0]:
df_track = spark.readStream.format("cloudFiles")\
          .option("cloudFiles.format", "parquet")\
          .option("cloudFiles.schemaLocation", "abfss://silver@anshulstorageproject.dfs.core.windows.net/DimTrack/checkpoint")\
          .load("abfss://bronze@anshulstorageproject.dfs.core.windows.net/DimTrack")

In [0]:
# Apply transformations
df_track_obj = reuseable()
df_track = df_track_obj.dropColumns(df_track, ['_rescued_data'])

# Deduplicate by track_id
df_track = df_track.dropDuplicates(['track_id'])

df_track = df_track.withColumn("durationFlag" , when(col('duration_sec')<150, "low").when(col('duration_sec')<300, "medium").otherwise("high"))
                                               
                                                               
df_track = df_track.withColumn("track_name" , regexp_replace(col('track_name'), '-', '  '))

In [0]:
df_track.writeStream.format("delta")\
    .outputMode("append")\
    .option("checkpointLocation", "abfss://silver@anshulstorageproject.dfs.core.windows.net/DimTrack/checkpoint")\
    .trigger(once=True)\
    .option("path", "abfss://silver@anshulstorageproject.dfs.core.windows.net/DimTrack/data")\
    .toTable("spotifycatalog.silver.DimTrack")

### **FactStream**

In [0]:
df_stream = spark.readStream.format("cloudFiles")\
          .option("cloudFiles.format", "parquet")\
          .option("cloudFiles.schemaLocation", "abfss://silver@anshulstorageproject.dfs.core.windows.net/FactStream/checkpoint")\
          .load("abfss://bronze@anshulstorageproject.dfs.core.windows.net/FactStream")

In [0]:
# Apply transformations
df_stream_obj = reuseable()
df_stream = df_stream_obj.dropColumns(df_stream, ['_rescued_data'])

In [0]:
df_stream.writeStream.format("delta")\
    .outputMode("append")\
    .option("checkpointLocation", "abfss://silver@anshulstorageproject.dfs.core.windows.net/FactStream/checkpoint")\
    .trigger(once=True)\
    .option("path", "abfss://silver@anshulstorageproject.dfs.core.windows.net/FactStream/data")\
    .toTable("spotifycatalog.silver.FactStream")